<div align="center">
    <h3>A Python API for the National Data Buoy Center</h3>
</div>


The National Oceanic and Atmospheric Association's National Data Buoy Center maintains marine monitoring and observation stations around the world[^1]. These stations report atmospheric, oceanographic, and other meterological data at regular intervals to the NDBC. Measurements are made available over HTTP through the NDBC's data service.

The ndbc-api is a python library that makes this data more widely accessible.

The ndbc-api is primarily built to parse whitespace-delimited oceanographic and atmospheric data distributed as text files for available time ranges, on a station-by-station basis[^2]. Measurements are typically distributed as `utf-8` encoded, station-by-station, fixed-period text files. More information on the measurements and methodology are available [on the NDBC website](https://www.ndbc.noaa.gov/docs/ndbc_web_data_guide.pdf)[^3].

[^1]: https://www.ndbc.noaa.gov/
[^2]: https://www.ndbc.noaa.gov/obs.shtml
[^3]: https://www.ndbc.noaa.gov/docs/ndbc_web_data_guide.pdf

This sample notebook covers some of the core functionality of the `NdbcApi`, including usage examples for retrieving the list of stations, station metadata, station measurements, and finding the nearest station to a given location.

We assume some familiarity with NDBC data buoy's, their purpose, and their data formats, but the examples help provide extra context for users new to the NDBC data service.  One important feature to consider is that not all data buoys are alike.  Some buoys provide a full suite of measurements, while others provide only a subset.  The `NdbcApi` provides methods to help users determine which measurements are available for a given station, and to retrieve those measurements.

### Setup

These setup steps are useful when cloning the project from source and running the notebooks locally.  If you are running the notebooks in a cloud environment, you can skip these steps and just run the cells under the "API Overview" heading.

In [1]:
import os
import sys

In [2]:
os.chdir("..")
sys.path.append(os.getcwd())

### API Overview

The API surface is exposed through the `NdbcApi` class.  The `NdbcApi` is a singleton, such that the underlying `RequestHandler` and NDBC station-level `RequestCache`s are shared between instances. Both the singleton metaclass and `RequestHandler` are implemented to reduce the likelihood of repeat requests to the NDBC's data service, and to converse NDBC resources. This is balanced by a station-level `cache_limit`, implemented as an LRU cache, which seeks to balance a respect for user resources and NDBC resources.

In [3]:
from ndbc_api import NdbcApi

In [4]:
api = NdbcApi()

#### Obtain a list of NDBC-maintained stations (data buoys)

Now that we have our `api` instance, we can begin to obtain information from the NDBC data service.

We begin by retrieving a list of all data buoys, and some of their high-level metadata.

In [5]:
api.stations()

,Station,Lat,Lon,Elevation,Name,Owner,Program,Type,Includes Meteorology,Includes Currents,Includes Water Quality,DART Program
0,0y2w3,44.794,-87.313,179.0,"Sturgeon Bay CG Station, WI",U.S.C.G. Marine Reporting Stations,IOOS Partners,fixed,False,False,False,False
1,13001,12.000,-23.000,0.0,NE Extension,Prediction and Research Moored Array in the At...,International Partners,buoy,True,False,False,False
2,13002,21.000,-23.000,0.0,NE Extension,Prediction and Research Moored Array in the At...,International Partners,buoy,True,False,False,False
3,13008,15.000,-38.000,0.0,Reggae,Prediction and Research Moored Array in the At...,International Partners,buoy,True,False,False,False
4,13009,8.000,-38.000,0.0,Lambada,Prediction and Research Moored Array in the At...,International Partners,buoy,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1340,yata2,59.548,-139.733,<NA>,"9453220 - Yakutat, Yakutat Bay, AK",NOS,NOS/CO-OPS,fixed,True,False,False,False
1341,ygnn6,43.262,-79.064,73.0,"Niagara Coast Guard Station, NY",NWS Eastern Region,IOOS Partners,fixed,True,False,False,False
1342,yktv2,37.227,-76.479,3.7,"8637689 - Yorktown USCG Training Center, VA",NOS,NOS/CO-OPS,fixed,True,False,False,False
1343,yrsv2,37.414,-76.712,11.0,"Taskinas Creek, Chesapeake Bay Reserve, VA",National Estuarine Research Reserve System,NERRS,fixed,True,False,False,False


It is important to note that the list of data buoys above is not exhaustive. The NDBC maintains a list of all active buoys [here](https://www.ndbc.noaa.gov/activestations.xml). The `get_stations` method returns a list of all active buoys maintained by the NDBC.

There are additional buoys which are supported by other programs and feed in to the NDBC data service. As such, they should be supported. For an example, consider station id `APAM2`, which is maintained by the National Ocean Service, but is available under the `NdbcApi`'s `get_data` method. If you would like to see a buoy or buoy family added to the API, please open an issue on the [GitHub repository](https:///www.github.com/cdjellen//ndbc-api).

#### Check the API's supported data formats

The `NbdcApi` seeks to enable easy access to data from most common models of data buoy and oceanographic station.

Data are assumed to follow common patterns based on the measurement type and the station type.  For example, a buoy that measures wind speed and direction will have a `cwind` mode, and a coastal station that measures wind speed and direction will have a `stdmet` mode.  The `NbdcApi` provides a `get_modes` method to list the available modes for a given station, and a `get_data` method to retrieve data for a given station and mode.

Not all stations have support for all modes.

In [6]:
api.get_modes()

['adcp',
 'cwind',
 'ocean',
 'spec',
 'stdmet',
 'supl',
 'swden',
 'swdir',
 'swdir2',
 'swr1',
 'swr2']

The `mode` parameter is ubiquitous in the `NdbcApi` (and of course in the upstream NDBC Data Service). It is used to specify the type of data to retrieve. As the indicators are not always intuitive, a descriptor of each mode is provided below:

* `adcp`: Acoustic Doppler Current Profiler measurements, providing information about ocean currents at different depths.
* `cwind`: Continuous winds data, offering high-frequency wind speed and direction measurements.
* `ocean`: Oceanographic data, including water temperature, salinity, and wave measurements.
* `spec`: Spectral wave data, providing detailed information about wave energy and direction.
* `stdmet`: Standard meteorological data, including air temperature, pressure, wind speed, and visibility.
* `supl`: Supplemental measurements, which can vary depending on the specific buoy and its sensors.
* `swden`: Spectral wave density data, providing information about the distribution of wave energy across different frequencies.
* `swdir`: Spectral wave direction data, indicating the primary direction of wave energy.
* `swdir2`: Secondary spectral wave direction data, capturing additional wave direction information.
* `swr1`: First-order spectral wave data, providing basic wave height and period information.
* `swr2`: Second-order spectral wave data, offering more detailed wave measurements.

These modes are based on the NDBC's data service, and while they cover much of the data they are not exhaustive. For a full list of modes, please refer to the [NDBC documentation](https://www.ndbc.noaa.gov/docs/ndbc_web_data_guide.pdf).

The API also now supports NetCDF4 data obtained through the [NOAA THREDDS](https://dods.ndbc.noaa.gov/thredds/catalog.html). Not all data is available through this server, but the subset which is can be obtained by passing `user_opendap=True` to the `get_modes` method.

In [7]:
api.get_modes(use_opendap=True)

['adcp', 'cwind', 'ocean', 'pwind', 'stdmet', 'swden', 'wlevel']

#### Find the nearest station to a location

In some cases, we might be interested in obtaining data from a location without a-priori knowledge of which data buoys or oceanography stations are nearby.  We can use the `get_nearest_stations` function to find the nearest stations to a given location.  This function returns the `station_id` of the nearest station to any given lat-lon location.

It is important to check the metadata for that station to ensure it is close enough to meet your needs.

In [8]:
api.nearest_station(lat="38.88N", lon="76.43W")

'tplm2'

We can also search for a station from floating-point longitude and latitude values.

In [9]:
api.nearest_station(lat=38.88, lon=-76.43)

'tplm2'

The API also supports a `radial_search` method, which returns all stations within a given radius of a given location. The radius is specified as a floating point value, with the units argument specifying whether the radius is in miles. kilometers or nautical miles. 

In [10]:
api.radial_search(lat="38.88N", lon="76.43W", radius=500, units="km")

,Station,Lat,Lon,Elevation,Name,Owner,Program,Type,Includes Meteorology,Includes Currents,Includes Water Quality,DART Program,distance
1280,tplm2,38.899,-76.436,0.0,"Thomas Point, MD",NDBC,NDBC Meteorological/Ocean,fixed,True,False,False,False,2.177979
213,44063,38.963,-76.448,0.0,"Annapolis, MD",Chesapeake Bay Interpretive Buoy System (CBIBS),IOOS Partners,buoy,False,False,False,False,9.369909
621,apam2,38.983,-76.479,1.4,"8575512 - Annapolis, MD",NOS,NOS/CO-OPS,fixed,True,False,False,False,12.225590
730,cpvm2,38.995,-76.388,3.4,"8575437 - Chesapeake Bay Bridge Visibility, MD",NOAA NOS PORTS,NOS/CO-OPS,fixed,True,False,False,False,13.307985
676,bslm2,38.781,-76.708,0.3,"Jug Bay, Chesapeake Bay Reserve, MD",National Estuarine Research Reserve System,NERRS,fixed,True,False,False,False,26.506690
...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,cblo1,41.981,-80.556,177.0,"Conneaut Breakwater Light, OH",NWS Eastern Region,IOOS Partners,fixed,True,False,False,False,491.192873
270,45142,42.740,-79.290,174.0,Port Colborne,Environment and Climate Change Canada,International Partners,buoy,False,False,False,False,492.561301
239,44402,39.314,-70.717,0.0,SOUTHEAST BLOCK CANYON - 130 NM SE of Fire Isl...,NDBC,Tsunami,dart,False,False,False,True,495.825675
1179,rprn6,43.263,-77.598,75.0,"Rochester Coast Guard, NY",NWS Eastern Region,IOOS Partners,fixed,True,False,False,False,497.635930


#### Obtain that station's metadata

The NDBC records some features of data buoys and oceanographic stations such as the location, station type, and elevations of various instruments.

Using the `station_id` obtained above, we can query the NDBC data service for some additional details about the nearest station to `lat='38.88N', lon='76.43W'`.  This is both to verify that the nearest station is indeed close to our desired location, and to learn more about how measurements of interest are collected.

In [11]:
api.station(station_id="tplm2")

{'Sea temp depth': '1 m below MLLW',
 'Barometer elevation': '12.2 m above mean sea level',
 'Anemometer height': '18 m above site elevation',
 'Air temp height': '17.4 m above site elevation',
 'Site elevation': '0 m above mean sea level',
 'Location': '38.899 N 76.436 W (38°53\'56" N 76°26\'9" W)',
 'Statation Type': 'Owned and maintained by National Data Buoy Center, C-MAN Station, MARS payload',
 'Name': 'Station TPLM2  - Thomas Point, MD'}

We can also obtain this metadata as a pandas DataFrame.

In [12]:
api.station(station_id="tplm2", as_df=True)

,0
Sea temp depth,1 m below MLLW
Barometer elevation,12.2 m above mean sea level
Anemometer height,18 m above site elevation
Air temp height,17.4 m above site elevation
Site elevation,0 m above mean sea level
Location,"38.899 N 76.436 W (38°53'56"" N 76°26'9"" W)"
Statation Type,Owned and maintained by National Data Buoy Cen...
Name,"Station TPLM2 - Thomas Point, MD"


While this information provides some helpful context about the station, it does not tell us what measurements are actually collected at the station.  We learned that it is a C-MAN station, and that it is maintained by the NDBC.  In order to determine what realtime and historical measurements are available for query, we can make two additional API calls.

#### Obtain the realtime measurements available at that station

Each station, due to the variety of buoy designs and environmental factors, offers a potentially different set of available measurements.  In order to determine what realtime measurements are available for a specific station, we can use the `available_realtime` API method.

In [13]:
api.available_realtime(station_id="tplm2")

['cwind', 'stdmet']

While understanding the set of currently-supported measurements is important for data retrieval, it is also useful to understand the historical measurements available at a station.  The `available_historical` method provides this data in the same format as above, indexed by year.

For both realtime and historical data, the API supports returning the response as pandas DataFrame when passing the keyword argument `as_df=True` to `available_realtime` or `available_historical`.

#### Obtain measurements over a given time range

Now that we know both `stdmet` (Standard Meterological) and `cwind` (Continuous Winds) data are available for our station, we can begin to obtain these measurements using our API.

We can select any time range of interest, regardless of the 45-day limit for "realtime" data before it becomes "historical" data.  The API abstracts these concerns away, and will automatically retrieve the data from the appropriate source, unifying it into a single `pd.DataFrame` or `dict` object.

We can begin by querying all `stdmet` data for our station `'tplm2'` for calendar-year 2020.  These dates follow the time conventions of the NDBC data service.

In [14]:
df_stdmet_tplm2 = api.get_data(
    'tplm2',
    'stdmet',
    '2020-01-01',
    '2022-01-01',
    as_df=True
)

By inspecting the returned `pd.DataFrame`, we can see that some of the typical `stdmet` feautures are unavailable for this station.  If we were to pull this data directly from the NDBC data service, these missing measurements would be marked with `99.0` `999` or `999.0` values (depending on the measurement).  However, the `stdmet` data service replaces these values with `NaN` values.  This is done to make it easier to work with the data in downstream applications and analyses.

In [15]:
df_stdmet_tplm2.head(3).T

timestamp,2020-01-01 00:00:00,2020-01-01 01:00:00,2020-01-01 02:00:00
station_id,tplm2,tplm2,tplm2
WDIR,188.0,273.0,286.0
WSPD,6.1,6.0,4.7
GST,6.2,6.9,5.6
WVHT,NaN,NaN,NaN
DPD,NaN,NaN,NaN
APD,NaN,NaN,NaN
MWD,NaN,NaN,NaN
PRES,1006.1,1006.8,1007.3
ATMP,9.5,9.3,8.7


In [16]:
df_stdmet_tplm2.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 17280 entries, (Timestamp('2020-01-01 00:00:00'), 'tplm2') to (Timestamp('2022-01-01 00:00:00'), 'tplm2')
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   WDIR    17255 non-null  float64
 1   WSPD    17280 non-null  float64
 2   GST     17280 non-null  float64
 3   WVHT    0 non-null      float64
 4   DPD     0 non-null      float64
 5   APD     0 non-null      float64
 6   MWD     0 non-null      float64
 7   PRES    17278 non-null  float64
 8   ATMP    17278 non-null  float64
 9   WTMP    17280 non-null  float64
 10  DEWP    14715 non-null  float64
 11  VIS     0 non-null      float64
 12  TIDE    0 non-null      float64
dtypes: float64(13)
memory usage: 2.4+ MB


In the informational view above, note that we have indexed the data by its `timestamp`, which was computed using the `'YY'`, `'MM'`, `'DD'`, `'hh'`, and `'mm'` fields typical of the NDBC data service.

We can also capture this data as a basic python `dict` object, which can be used to create other data structures or serialized for outside analysis or tools.

In [17]:
api.get_data(
    'tplm2',
    'stdmet',
    '2020-01-01',
    '2020-01-02',
    as_df=False
)

{'stdmet': [{'WDIR': {Timestamp('2020-01-01 00:00:00'): 188.0,
    Timestamp('2020-01-01 01:00:00'): 273.0,
    Timestamp('2020-01-01 02:00:00'): 286.0,
    Timestamp('2020-01-01 03:00:00'): 278.0,
    Timestamp('2020-01-01 04:00:00'): 293.0,
    Timestamp('2020-01-01 05:00:00'): 297.0,
    Timestamp('2020-01-01 06:00:00'): 296.0,
    Timestamp('2020-01-01 07:00:00'): 287.0,
    Timestamp('2020-01-01 08:00:00'): 287.0,
    Timestamp('2020-01-01 09:00:00'): 282.0,
    Timestamp('2020-01-01 10:00:00'): 286.0,
    Timestamp('2020-01-01 11:00:00'): 276.0,
    Timestamp('2020-01-01 12:00:00'): 271.0,
    Timestamp('2020-01-01 13:00:00'): 275.0,
    Timestamp('2020-01-01 14:00:00'): 266.0,
    Timestamp('2020-01-01 15:00:00'): 269.0,
    Timestamp('2020-01-01 16:00:00'): 270.0,
    Timestamp('2020-01-01 17:00:00'): 260.0,
    Timestamp('2020-01-01 18:00:00'): 252.0,
    Timestamp('2020-01-01 19:00:00'): 257.0,
    Timestamp('2020-01-01 20:00:00'): 271.0,
    Timestamp('2020-01-01 21:00:00'):

Some research efforts require obtaining data from the same mode across many stations, multiple modes for the same station, or multiple modes across multiple stations. These use cases are also supported by the `get_data` method. For these use cases, `station_ids` (a list of station ids) or `modes` can be specified in lieu of `station_id` or `mode`. Please use key-word arguments (specifying the parameter name directly) to leverage these features.

In [18]:
df_annapolis_met = api.get_data(
    station_ids=['tplm2', 'apam2', 44063, 'CPVM2'],
    modes=['stdmet'],
    start_time='2023-01-01',
    end_time='2024-01-01',
    as_df=True
)

The returned `pd.DataFrame` will match the format explored earlier, with one key difference. The `station_id` is now included along with the `timestamp` as a multi-index.

All station ids included in `station_ids` are stored in the same long-formatted `pd.DataFrame`, and can be filtered based on the feature of interest (e.g. `'WDIR'`), the station_id (e.g. `TPLM2`), and the timestamp.

The Pandas API specifications suggests using the `xs` method to filter data based on the multi-index.  This method is demonstrated below.

In [19]:
df_annapolis_met.xs('tplm2', level='station_id').head(3).T

timestamp,2023-01-01 00:00:00,2023-01-01 01:00:00,2023-01-01 02:00:00
WDIR,141.0,182.0,172.0
WSPD,6.9,5.5,6.5
GST,7.2,5.7,6.7
WVHT,NaN,NaN,NaN
DPD,NaN,NaN,NaN
APD,NaN,NaN,NaN
MWD,NaN,NaN,NaN
PRES,1011.3,1011.5,1011.8
ATMP,10.5,10.1,8.5
WTMP,NaN,NaN,NaN


As above, can also capture this data as a basic python `dict` object. As no all stations in a given `get_data` request might have all requested `modes`, we log errors in cases where no data is found for a given `(station_id, mode)` pair. We return all data available, noting the error through api logging.

In [20]:
api.get_data(
    station_ids=['tplm2', 'apam2'],
    modes=['stdmet', 'cwind'],  # cwind is not available for 'apam2' so we will log an error and continue
    start_time='2023-01-01',
    end_time='2024-01-01',
    as_df=False
)

{'stdmet': [{'WDIR': {Timestamp('2023-01-01 00:00:00'): 141.0,
    Timestamp('2023-01-01 01:00:00'): 182.0,
    Timestamp('2023-01-01 02:00:00'): 172.0,
    Timestamp('2023-01-01 03:00:00'): 87.0,
    Timestamp('2023-01-01 04:00:00'): 224.0,
    Timestamp('2023-01-01 05:00:00'): 239.0,
    Timestamp('2023-01-01 06:00:00'): 240.0,
    Timestamp('2023-01-01 07:00:00'): 238.0,
    Timestamp('2023-01-01 08:00:00'): 243.0,
    Timestamp('2023-01-01 09:00:00'): 263.0,
    Timestamp('2023-01-01 10:00:00'): 219.0,
    Timestamp('2023-01-01 11:00:00'): 277.0,
    Timestamp('2023-01-01 12:00:00'): 274.0,
    Timestamp('2023-01-01 13:00:00'): 265.0,
    Timestamp('2023-01-01 14:00:00'): 273.0,
    Timestamp('2023-01-01 15:00:00'): 271.0,
    Timestamp('2023-01-01 16:00:00'): 279.0,
    Timestamp('2023-01-01 17:00:00'): 278.0,
    Timestamp('2023-01-01 18:00:00'): 277.0,
    Timestamp('2023-01-01 19:00:00'): 104.0,
    Timestamp('2023-01-01 20:00:00'): 146.0,
    Timestamp('2023-01-01 21:00:00'): 

##### Native `NetCDF4` data through the NDBC DODS/THREDDS data service

The NDBC API now supports retrieving data across stations and modes in the native `DODS NetCDF4` [format](https://dods.ndbc.noaa.gov/) provided by [THREDDS](https://dods.ndbc.noaa.gov/thredds/catalog/data/catalog.html). This data is retrieved from the DODS server through HTTPS, with the resulting data matching the quality-controlled records as closely as possible.

Data retrieval through THREDDS is controlled using the `as_xarray_dataset` argument (defaults to `False`).

As with the standard usage of the `get_data` method, the `start_time`, `end_time`, and `cols` arguments can be used to efficiently filter data during retrieval.

In [21]:
ds = api.get_data(station_ids=["tplm2"], modes=["stdmet"], start_time="2019-06-01", end_time="2024-06-01", as_xarray_dataset=True)

In [22]:
ds

<xarray.Dataset> Size: 3MB
Dimensions:                  (time: 39874, latitude: 1, longitude: 1)
Coordinates:
  * time                     (time) datetime64[ns] 319kB 2019-06-01 ... 2023-...
  * latitude                 (latitude) float32 4B 38.9
  * longitude                (longitude) float32 4B -76.44
Data variables: (12/13)
    wind_dir                 (time, latitude, longitude) float64 319kB 92.0 ....
    wind_spd                 (time, latitude, longitude) float32 159kB 1.8 .....
    gust                     (time, latitude, longitude) float32 159kB 1.9 .....
    wave_height              (time, latitude, longitude) float32 159kB nan .....
    dominant_wpd             (time, latitude, longitude) timedelta64[ns] 319kB ...
    average_wpd              (time, latitude, longitude) timedelta64[ns] 319kB ...
    ...                       ...
    air_pressure             (time, latitude, longitude) float32 159kB 1.01e+...
    air_temperature          (time, latitude, longitude) float32 159kB 25.5 ....
    sea_surface_temperature  (time, latitude, longitude) float32 159kB 23.7 ....
    dewpt_temperature        (time, latitude, longitude) float32 159kB 17.2 ....
    visibility               (time, latitude, longitude) float32 159kB nan .....
    water_level              (time, latitude, longitude) float32 159kB nan .....
Attributes:
    institution:  NOAA National Data Buoy Center and Participators in Data As...
    url:          http://dods.ndbc.noaa.gov
    quality:      Automated QC checks with manual editing and comprehensive m...
    conventions:  COARDS
    station:      tplm2
    comment:      Thomas Point, MD
    location:     38.899 N 76.436 W

The returned `netCDF4.Dataset` objects are ephemeral, but can be explicitly saved to disk by passing the Dataset and a file path to the API's `save_netcdf_dataset` method, as below:

In [23]:
api.save_xarray_dataset(ds, "test.nc")

The `save_xarray_dataset` method will save the dataset to disk in the specified file path. The file path should include the desired file name and extension (e.g. `my_dataset.nc`).

The method also supports keyword arguments, which are passed to the underlying `xarray.Dataset`'s `to_netcdf` method. This allows for additional control over the saved dataset, such as compression level, encoding, and other options. For example, to use the `h5netcdf` backend when saving the dataset, one could specify `kwargs={'engine': 'h5netcdf'}`.

### Logging Configuration

The API supports logging configuration at initialization through two arguments:

```python3
api = NdbcApi(
    logging_level=logging.WARNING,  # one of logging.DEBUG, logging.INFO, logging.WARNING, or logging.ERROR
    filename=None,  # an optional filepath at which to write log files
    ...
)
```

At any time, the logging can be updated by calling `api.configure_logging(level=..., filename=...)` as shown below.


In [24]:
import logging

api.configure_logging(level=logging.DEBUG)

In [25]:
api.get_data(
    station_ids=['tplm2', 'apam2'],
    modes=['stdmet', 'cwind'],  # cwind is not available for 'apam2' so we will log an error and continue
    start_time='2023-01-01',
    end_time='2024-01-01',
)

[DEBUG]: {'message': "`get_data` called with arguments: {'self': <ndbc_api.ndbc_api.NdbcApi object at 0x116ff78b0>, 'station_id': None, 'mode': None, 'start_time': '2023-01-01', 'end_time': '2024-01-01', 'use_timestamp': True, 'as_df': True, 'cols': None, 'station_ids': ['tplm2', 'apam2'], 'modes': ['stdmet', 'cwind'], 'as_xarray_dataset': False, 'use_opendap': None}"}
{'message': "`get_data` called with arguments: {'self': "
            "<ndbc_api.ndbc_api.NdbcApi object at 0x116ff78b0>, 'station_id': "
            "None, 'mode': None, 'start_time': '2023-01-01', 'end_time': "
            "'2024-01-01', 'use_timestamp': True, 'as_df': True, 'cols': None, "
            "'station_ids': ['tplm2', 'apam2'], 'modes': ['stdmet', 'cwind'], "
            "'as_xarray_dataset': False, 'use_opendap': None}"}
[INFO]: {'message': "Processing request for station_ids ['tplm2', 'apam2'] and modes ['stdmet', 'cwind']"}
{'message': "Processing request for station_ids ['tplm2', 'apam2'] and modes "
    

,,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE,GDR,GTIME
timestamp,station_id,,,,,,,,,,,,,,,
2023-01-01 00:00:00,tplm2,141.0,6.9,7.2,NaN,NaN,NaN,NaN,1011.3,10.5,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 01:00:00,tplm2,182.0,5.5,5.7,NaN,NaN,NaN,NaN,1011.5,10.1,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 02:00:00,tplm2,172.0,6.5,6.7,NaN,NaN,NaN,NaN,1011.8,8.5,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 03:00:00,tplm2,87.0,0.4,0.7,NaN,NaN,NaN,NaN,1010.6,7.4,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 04:00:00,tplm2,224.0,3.6,3.7,NaN,NaN,NaN,NaN,1011.1,12.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 23:20:00,tplm2,140.0,5.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 23:30:00,tplm2,144.0,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 23:40:00,tplm2,142.0,5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Concluding remarks

The NDBC API is a living project, and is open both to community requests and code contributions.  Please feel free to submit a pull request or open an issue on the [GitHub repository](https://www.github.com/cdjellen/ndbc-api) if you have any questions or suggestions.

Thank you for your time and have an excellent rest of your day, wherever in the world you are!